# Findings Notebook 2 - Further Analysis of Features

This notebook contains a further analysis of the features 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import re
import glob
import math
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pyspark.sql.functions as F
import sklearn
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime, timedelta
from collections import defaultdict
from multiprocessing import Manager

In [2]:
spark = (
    SparkSession.builder.appName('Feature Findings 2')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '7g')
    .config('spark.executor.memory', '3g')
    .getOrCreate()
)

24/10/18 21:30:23 WARN Utils: Your hostname, Alans-MacBook-Air-4.local resolves to a loopback address: 127.0.0.1; using 192.168.0.52 instead (on interface en0)
24/10/18 21:30:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/18 21:30:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
sdf = spark.read.parquet(".././data/curated/transactions_preprocessed.parquet")
df = spark.read.parquet('.././data/curated/filtered')
df.show()

+------------+--------------------+---------+----+------------------+-----------+------------------+------------------+--------------------+-------------------+-------------------+----------------------+----------------+------------------------+---------------+
|merchant_abn|                tags|take_rate|type|     Total Revenue|Total Sales|    Merchant Fraud|    Consumer Fraud|Median Family Income|avg(seasonal_sales)|avg(original_sales)|Average Household Size|Unique Customers|Average Expected Revenue|Days with Sales|
+------------+--------------------+---------+----+------------------+-----------+------------------+------------------+--------------------+-------------------+-------------------+----------------------+----------------+------------------------+---------------+
| 70217388552|gift, card, novel...|     5.52|   a| 78993.54387353452|        885|30.579032159006303|13.917275422539495|  1971.5435028248587|  6545.891073446327|  6530.004406779661|     2.458870056497175|           

In [6]:
sdf.show()

24/10/18 17:39:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+----+--------+--------------+------------+-------+-----------+------------------+--------------------+-------------+-----------------+-----+------+------------------+--------------------+--------------------+----+---------+--------------------------+----------+--------------+----------------+----------------+------------------+------------------+-------------------------+-------------------------+----------------------+-------+-------+-------+--------------------+--------------------+-------------------+-----------+------------+-----------+----------+--------------+--------------+
|month|year|postcode|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|consumer_name|          address|state|gender| fraud_probability|       merchant_name|                tags|type|take_rate|merchant_fraud_probability|AREASQKM21|    SHAPE_Leng|      SHAPE_Area|             lon|               lat|Median_age_persons|Median_tot_fam_inc_weekly|Median_tot_hhd_inc_weekl

#### Days with Sales

In [8]:
pdf = df.pandas_api()
pdf['Days with Sales'].plot.density(bw_method = 1)

There are 606 days between February 28th 2021 and October 26th 2022 (inclusive)

#### Total Revenue

In [9]:
sdf.agg(F.max("order_datetime")).collect()[0][0] - sdf.agg(F.min("order_datetime")).collect()[0][0]

datetime.timedelta(days=605)

#### Unique Customers

In [10]:
pdf['Unique Customers'].plot.density(bw_method = 3)

The maximum amount earned with less than 30% days of sales is 872,493 which is 10% of the greatest amount 

#### Analyzing Number of Unique Customers

In [19]:
df.filter(F.col('Unique Customers') < 0.2*20111).sort('Total Revenue', ascending = False)

merchant_abn,tags,take_rate,type,Total Revenue,Total Sales,Merchant Fraud,Consumer Fraud,Median Family Income,avg(seasonal_sales),avg(original_sales),Average Household Size,Unique Customers,Average Expected Revenue,Days with Sales
82368304209,telecom,5.55,a,8213998.650508907,4324,31.52960834197179,11.954378094419296,1973.4174375578168,6413.756891766889,6395.690957446805,2.449329324699353,3860,62.743220816107886,606
31334588839,"antique, repairs,...",4.22,b,8146008.821656872,1284,32.33337286435618,20.673723709203585,1959.0366043613708,6571.395872274143,6579.63746105919,2.45031152647975,1238,126.04520154914597,540
79827781481,"furniture, home f...",6.82,a,8125906.617439892,3997,29.655087571235782,11.731592007701954,1978.805854390793,6434.07908431324,6424.116287215415,2.4469602201651233,3593,85.36021948546083,605
93260930990,telecom,3.15,b,7470121.383450534,2001,32.62367028963542,13.710157667136226,1994.0919540229886,6609.2,6609.501249375311,2.4735632183908045,1913,66.861698399791,581
31385641294,motor vehicle sup...,4.52,b,7078190.100018134,3206,29.571387727673752,12.607563446557382,1991.4875233936368,6492.677479725522,6462.7658764816,2.465190268247036,2958,60.312802126031556,603
19492220327,"jewelry, watch, c...",4.93,b,6635720.132917706,671,34.36047166825414,26.199649112828517,1944.7078986587182,6390.141430700446,6418.177198211625,2.4402384500745153,660,208.33887746644814,405
98166254020,"art dealers, gall...",5.96,a,6205605.750689492,3380,33.72512049377439,11.800082458034488,1970.4887573964497,6479.458520710066,6475.01381656805,2.456568047337278,3126,63.04547252862358,600
70009327857,"lawn, garden supp...",2.45,c,5485587.115477327,4143,28.68018049819,12.592316875758748,1983.5025343953657,6527.955756698049,6524.075838764179,2.456577359401401,3709,20.34452652329077,605
87084550311,"bicycle,",4.58,b,5140465.400699704,3996,40.722758418671965,12.055768709962742,1973.264014014014,6449.241766766771,6444.415390390391,2.4599599599599604,3641,30.877996488448325,604
90918180829,"jewelry, watch, c...",2.62,c,4795855.192886579,477,35.32904552241743,27.380691576135373,1951.5618448637317,6699.689937106918,6694.9408805031435,2.449266247379455,470,107.46339275875948,326


In [20]:
df.filter(F.col('Unique Customers') > 0.2*20111).sort('Total Revenue', ascending = False)

merchant_abn,tags,take_rate,type,Total Revenue,Total Sales,Merchant Fraud,Consumer Fraud,Median Family Income,avg(seasonal_sales),avg(original_sales),Average Household Size,Unique Customers,Average Expected Revenue,Days with Sales
39649557865,"artist supply, craft",1.47,c,8257372.182656345,18309,28.69020428605009,15.110408662014475,1978.72975039598,6490.639914795987,6482.051406412153,2.4591512370965054,12085,4.058066651157852,606
96680767841,motor vehicle sup...,5.91,a,8222773.144075594,26135,29.55524469042332,14.252019012039579,1979.6867419169696,6442.963191123008,6428.874918691434,2.4581901664434587,14641,11.258937854356368,606
27093785141,"stationery, offic...",2.73,c,8119405.467539969,21574,28.915730332861777,13.872033846849893,1979.7801056827664,6508.544029850738,6501.364169834079,2.4617827013998292,13218,6.279012894969426,606
90568944804,"tent, awning",4.1,b,8054447.097773365,8970,30.43953317851763,13.039956198951359,1978.4014492753622,6534.3725641025585,6514.560613154952,2.456889632107024,7273,22.56197843216561,606
32709545238,"stationery, offic...",3.04,c,8048827.8762068525,10701,28.900516479769344,13.53975160343134,1972.3323988412299,6498.441538173989,6500.527296514337,2.456219044949071,8348,14.122484016645002,606
50315283629,"lawn, garden supp...",1.76,c,8044038.10500395,24907,28.528115957711417,15.50495166945586,1979.0694182358372,6507.730461316089,6501.644357008086,2.4563977998153086,14299,3.4529075867876453,606
35909341340,computer programm...,4.8,b,8027391.329884788,31966,29.1330531100717,13.033167610485483,1978.6210348495276,6486.072999436922,6473.117327785794,2.4583213414252545,16006,7.425665317122769,606
38700038932,"tent, awning",6.31,a,7990772.979129689,5944,30.39533841000751,12.089089224381448,1983.4680349932705,6445.717513458952,6437.875471063257,2.4535834454912537,5154,52.204739820541256,606
86578477987,"watch, clock, jew...",6.43,a,7972145.046437768,227775,28.576456098300703,15.140652905079124,1977.0436966304467,6505.567195697163,6494.275517067175,2.4582956865327588,20111,1.3648947806370788,606
48534649627,"opticians, optica...",6.64,a,7894321.469880473,55666,28.572026824807107,14.921575149033098,1977.5257068946933,6520.805083893311,6514.311468400855,2.458285129163223,18881,5.711663727137508,606


In [21]:
df.filter(F.col('Unique Customers') > 0.2*20111).sort('Unique Customers', ascending = False)

merchant_abn,tags,take_rate,type,Total Revenue,Total Sales,Merchant Fraud,Consumer Fraud,Median Family Income,avg(seasonal_sales),avg(original_sales),Average Household Size,Unique Customers,Average Expected Revenue,Days with Sales
64203420245,"tent, awning",2.86,c,6299699.598407542,217571,30.612495680513543,14.871396677745812,1978.0475247160698,6483.831479838419,6475.44229377983,2.458449885324781,20111,0.489725251044011,606
86578477987,"watch, clock, jew...",6.43,a,7972145.046437768,227775,28.576456098300703,15.140652905079124,1977.0436966304467,6505.567195697163,6494.275517067175,2.4582956865327588,20111,1.3648947806370788,606
45629217853,"gift, card, novel...",6.98,a,7018507.434966652,190768,30.571603398126562,15.773599561837635,1974.7658831669883,6501.068451731659,6495.1992037447535,2.4591949383544414,20111,1.5099632925580027,606
89726005175,"tent, awning",6.01,a,7428298.547384338,180185,30.468602464836877,14.907350810485063,1976.4575353109303,6473.880059383172,6466.097259483214,2.4586203069067913,20110,1.4710167475270712,606
24852446429,florists supplies...,2.94,c,7254337.468013417,241506,52.77404521439999,15.138574786399698,1974.2637946883308,6499.6533651334585,6490.692043261773,2.4574702077794957,20110,0.35430937934354906,606
49891706470,"tent, awning",5.8,a,5993213.999709736,206878,30.484949771451102,14.911243830439796,1974.3533096801013,6493.379237037919,6483.0795763685455,2.4578790398205776,20109,0.9924495350497905,606
46804135891,"opticians, optica...",2.93,c,5871675.405216906,195655,28.876770729055476,14.995088166968143,1975.5919347831643,6495.082802381439,6486.2131312768515,2.456755002427747,20109,0.5305131766343497,606
80324045558,"gift, card, novel...",1.47,c,6037559.227590772,164419,50.46418613606594,14.487045677369357,1974.6811439067262,6503.3918865822425,6493.121574148852,2.459331950686982,20107,0.22908055802067176,606
43186523025,florists supplies...,4.47,b,7553315.176859005,167852,56.08790941390954,14.730184948009128,1975.4696220479946,6497.871535042539,6489.126602006412,2.458823249052751,20106,0.7526535003252506,606
68216911708,"computers, comput...",3.05,c,5816858.363979887,151084,30.02340244819238,14.935499820102965,1976.622938233036,6496.782822138482,6486.902405946259,2.4592253316036423,20099,0.6973622660511581,606


In [22]:
df.filter((F.col('tags') == 'telecom') & (F.col('Days with Sales')> 181)).sort('Total Revenue', ascending = False)

merchant_abn,tags,take_rate,type,Total Revenue,Total Sales,Merchant Fraud,Consumer Fraud,Median Family Income,avg(seasonal_sales),avg(original_sales),Average Household Size,Unique Customers,Average Expected Revenue,Days with Sales
82368304209,telecom,5.55,a,8213998.650508907,4324,31.52960834197179,11.954378094419296,1973.4174375578168,6413.756891766889,6395.690957446805,2.449329324699353,3860,62.743220816107886,606
93260930990,telecom,3.15,b,7470121.383450534,2001,32.62367028963542,13.710157667136226,1994.0919540229886,6609.2,6609.501249375311,2.4735632183908045,1913,66.861698399791,581
50589958690,telecom,3.15,b,2216162.3574642558,1772,32.704336225601736,11.551007648325578,2005.0220090293453,6524.900733634312,6527.014898419869,2.4601580135440178,1690,23.322256207284667,573
82545351270,telecom,3.24,b,1229552.9175486553,638,32.75218296900842,11.954402993575806,1957.6081504702195,6341.078996865203,6304.123354231974,2.4520376175548595,631,36.55102654431709,401
77990807356,telecom,3.95,b,1161682.0327630974,592,32.51830633026377,12.007762428021618,1992.106418918919,6446.797635135134,6455.721959459459,2.464527027027027,583,45.02798474751442,368
81836621910,telecom,1.85,c,1155797.7499907908,368,32.671140083617935,13.003550126730053,1950.7010869565217,6555.9366847826095,6582.6407608695645,2.4695652173913047,366,33.30066823748859,289
82044452251,telecom,1.05,d,1128070.078800251,886,32.8224792412314,11.803016019512745,1951.6286681715576,6428.532957110607,6465.1540632054175,2.4519187358916477,867,7.888434541530408,456
97860823526,telecom,5.85,a,1116323.1910591056,617,31.484648226063403,12.037697470564778,1979.418152350081,6587.676823338736,6569.021231766612,2.450405186385738,607,63.25991928399708,388
74874670528,telecom,1.99,c,1115775.307654166,352,32.699077842535395,13.023122606982593,1937.2272727272727,6760.211647727273,6778.977840909091,2.4468750000000004,351,36.06559475928188,271
72897360319,telecom,6.28,a,1053687.209145201,787,31.476380308428546,11.469207449086502,1979.8678526048284,6491.582337992375,6497.546759847522,2.474714104193138,767,50.529661458227885,433


Reason for not Using Unique Customers - Telecom which has less customers but high number of sales, less merchants sell telecom but they buy from it very often 

In [23]:
sdf.filter(F.col('tags') == 'telecom').show

<bound method DataFrame.show of +-----+----+--------+--------------+------------+-------+-----------+------------------+--------------------+----------------+--------------------+-----+-----------+------------------+--------------------+-------+----+---------+--------------------------+----------+---------------+----------------+------------------+-------------------+------------------+-------------------------+-------------------------+----------------------+-------+-------+-------+--------------------+--------------------+------------------+-----------+------------+-----------+----------+--------------+--------------+
|month|year|postcode|order_datetime|merchant_abn|user_id|consumer_id|      dollar_value|            order_id|   consumer_name|             address|state|     gender| fraud_probability|       merchant_name|   tags|type|take_rate|merchant_fraud_probability|AREASQKM21|     SHAPE_Leng|      SHAPE_Area|               lon|                lat|Median_age_persons|Median_tot_fam_

#### Further Analyzing Total Revenue

In [24]:
df.filter(F.col('Total Revenue') < 1000000).sort('Total Revenue', ascending = False).count()

3587

In [25]:
df.filter(F.col('Total Revenue') < 1000000).sort('Days with Sales', ascending = True)

merchant_abn,tags,take_rate,type,Total Revenue,Total Sales,Merchant Fraud,Consumer Fraud,Median Family Income,avg(seasonal_sales),avg(original_sales),Average Household Size,Unique Customers,Average Expected Revenue,Days with Sales
46537010521,"jewelry, watch, c...",2.18,c,25661.54568658274,1,36.16713055872844,51.328864293003456,1541.0,3873.2,3982.7,2.3,1,173.80215351172788,1
93267734067,"jewelry, watch, c...",2.57,c,16394.519414851562,1,40.75390612794177,39.70626262122254,2183.0,10928.0,10536.4,2.5,1,150.5094403985737,1
42461534060,"antique, repairs,...",3.98,b,22600.105672960628,1,28.04557596448634,47.31792066243643,1414.0,10457.6,9203.1,2.3,1,340.96825825447456,1
62688594508,"jewelry, watch, c...",0.88,d,21112.567917387372,1,37.986405931722075,35.780937947892255,1857.0,7788.6,7807.8,1.7,1,73.99026659595687,1
10404542215,"antique, repairs,...",6.93,a,22773.975280323484,1,32.98680171018991,60.48645653542324,1792.0,2034.1,2068.7,2.4,1,417.90580415788446,1
39150153670,"antique, repairs,...",6.99,a,38089.93927161727,1,44.86946087282027,69.24693782245694,2190.0,6418.6,6206.0,2.5,1,451.4067634155919,1
24212300522,"jewelry, watch, c...",4.04,b,16065.148211715352,1,42.49008491379519,37.925715872227904,2032.0,8840.7,8558.2,2.5,1,231.6970731845669,1
28311306642,"antique, repairs,...",4.93,b,21159.33121821739,1,32.56142976361132,45.31585993810361,2071.0,2215.5,2155.7,2.6,1,384.6968209152358,1
29068447069,"antique, repairs,...",3.28,b,73901.57508882221,2,26.474975193587735,72.20559678419119,3750.0,3586.7,3497.5,2.0,1,247.67950787261248,1
47047735645,"antique, repairs,...",1.54,c,40896.25782166824,1,40.69388326510747,67.93793282965282,2064.0,8701.9,8418.8,1.8,1,119.75545319937264,1


3175 merchants make less than 1000000

In [26]:
df.filter(F.col('Total Revenue') > 1000000).sort('Total Revenue', ascending = False).count()

439

#### Studying Relationships Between Different Features

In [27]:
import plotly.express as px

In [28]:
dff = df.toPandas()

In [29]:
fig = px.scatter(dff, y = 'Total Revenue', x = 'Unique Customers',  hover_data=['merchant_abn'], color_discrete_sequence=['purple'], width = 500, height = 550, title='Unique Customers and Revenue for Merchants')
fig.update_layout(title_x = 0.5)
fig.show()

In [30]:
pdf.plot.scatter('Total Revenue', 'Merchant Fraud', c = df['tags'], title = 'Merchant Fraud vs Total Revenue')

In [31]:
fig = px.scatter(dff, x = 'Total Revenue', y = 'Merchant Fraud', color = 'tags' , hover_data=['merchant_abn'], title = 'Merchant Fraud vs Total Revenue')
fig.show()

In [32]:
sdf.filter((F.col('tags') == 'gift, card, novelty, souvenir') & (F.col('merchant_fraud_probability') > 50.000)).count()

387974

In [33]:
sdf.filter((F.col('tags') == 'gift, card, novelty, souvenir')).count()

1369641

In [34]:
pdf.plot.scatter('Total Revenue', 'Consumer Fraud', c = df['tags'], title = 'Consumer Fraud and Unique Customers')

In [35]:
fig = px.scatter(dff, x = 'Total Revenue', y = 'Consumer Fraud', color = 'tags' , hover_data=['merchant_abn'], title = 'Consumer Fraud and Unique Customers')
fig.show()

In [36]:
pdf.plot.scatter('Total Revenue', 'Days with Sales', c = df['tags'], title = 'Merchant Revenue and Days they have Sales')

In [37]:
fig2 = px.scatter(dff, x = 'Total Revenue', y = 'Median Family Income',hover_data=['merchant_abn'], color_discrete_sequence= ['pink'] , title  = 'Average Consumer Median Family Income and Revenue by Merchants')
fig2.show()

In [38]:
fig1 = px.scatter(dff, x = 'Total Revenue', y = 'Average Household Size' , hover_data=['merchant_abn'], color_discrete_sequence= ['green'], title  = 'Average Consumer Household Size and Revenue by Merchants')
fig1.show()

In [39]:
sdf.filter(F.col('merchant_abn') == 17678252870).show(truncate = False)

+-----+----+--------+--------------+------------+-------+-----------+------------------+------------------------------------+----------------+------------------------------+-----+-----------+------------------+-------------------------+----------------------------+----+---------+--------------------------+----------+---------------+----------------+------------------+-------------------+------------------+-------------------------+-------------------------+----------------------+-------+-------+-------+--------------------+--------------------+-------------------+-----------+------------+-----------+----------+--------------+--------------+
|month|year|postcode|order_datetime|merchant_abn|user_id|consumer_id|dollar_value      |order_id                            |consumer_name   |address                       |state|gender     |fraud_probability |merchant_name            |tags                        |type|take_rate|merchant_fraud_probability|AREASQKM21|SHAPE_Leng     |SHAPE_Area      |

#### Analyzing Health and Beauty Spas

They appear to have a higher fraud value as a group compared to other tags. 

In [40]:
sdf.filter(F.col('tags') == 'health, beauty spas').sort('merchant_abn', 'consumer_name')

month,year,postcode,order_datetime,merchant_abn,user_id,consumer_id,dollar_value,order_id,consumer_name,address,state,gender,fraud_probability,merchant_name,tags,type,take_rate,merchant_fraud_probability,AREASQKM21,SHAPE_Leng,SHAPE_Area,lon,lat,Median_age_persons,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,Average_household_size,Tot_M,Tot_P,Tot_F,total_original_sales,total_seasonal_sales,expected_revenue,day_of_week,day_of_month,day_of_year,is_holiday,seasonal_sales,original_sales
6,2021,880,2021-06-19,11199348915,20818,1111912,774.7961615267519,9a4d6c4a-0e1f-4b0...,Aaron Fitzpatrick,237 Paige Forges,NT,Male,14.129550456214693,Orci Luctus PC,"health, beauty spas",b,4.1,29.33704721181367,88.6672,0.990311050003,0.00736664454331,136.64584246933453,-12.246116763360021,32.0,2425.0,2406.0,3.3,2477.0,4953.0,2478.0,29433.8,30529.4,19.2755524919903,7,19,170,0,292.8,304.2
5,2022,4020,2022-05-14,11199348915,15430,487488,989.2342076575333,65a23004-7e22-45f...,Aaron Garcia,7230 Loretta Street,QLD,Undisclosed,12.937492700381771,Orci Luctus PC,"health, beauty spas",b,4.1,68.27843632543912,11.7011,0.20048422692,0.00106599898515,153.1021513405475,-27.214364468324263,50.0,1914.0,1365.0,2.2,12183.0,25603.0,13418.0,33287.1,34012.1,11.201308020602626,7,14,134,0,7083.5,6846.3
6,2021,4800,2021-06-02,11199348915,17290,172769,228.55584797659293,d7a0114b-657d-414...,Aaron Santana,45069 Sullivan Cove,QLD,Undisclosed,12.989583785556885,Orci Luctus PC,"health, beauty spas",b,4.1,68.27843632543912,2435.9895,4.55917283702,0.210807924998,148.52292437891708,-20.404600192555616,44.0,1861.0,1489.0,2.5,5181.0,10081.0,4899.0,29433.8,30529.4,2.586437735534673,4,2,153,0,6399.1,6164.1
3,2022,6017,2022-03-05,11199348915,16184,317856,263.0082324789289,ca0c23df-73c9-4ae...,Aaron Waller,96419 Alicia Lane...,WA,Male,15.180893990535417,Orci Luctus PC,"health, beauty spas",b,4.1,29.005493581526984,8.0995,0.160928246253,7.72308881745E-4,115.81257121552099,-31.91080499083522,34.0,1965.0,1475.0,1.9,2226.0,4507.0,2281.0,32830.9,33559.6,6.493392188422599,7,5,64,0,3676.6,3590.8
9,2021,5117,2021-09-13,11199348915,1453,352530,835.5041396141315,d037bb71-f90d-45a...,Adam Carpenter,5088 Michael Dale,SA,Male,13.871511819510296,Orci Luctus PC,"health, beauty spas",b,4.1,28.956947892226466,14.0543,0.224721948449,0.00138177386096,138.64976540648289,-34.63857839111548,35.0,2365.0,2248.0,3.1,2030.0,4051.0,2025.0,28901.7,29696.6,20.9604642658853,2,13,256,0,2046.8,1981.3
10,2022,3463,2022-10-13,11199348915,7858,1058829,604.3304017625977,a52f4b15-632a-4a5...,Adam Heath,85792 Carey Stree...,VIC,Male,13.650038965958148,Orci Luctus PC,"health, beauty spas",b,4.1,57.59302819113623,530.6538,1.69548567694,0.0536737185036,144.00893634206358,-36.93446951601609,57.0,1525.0,1127.0,2.1,1276.0,2588.0,1317.0,35496.2,35150.0,9.073141977487177,5,13,286,0,9056.5,9049.7
11,2021,6149,2021-11-04,11199348915,14605,649177,1173.1293350191165,09a793f8-c98c-47e...,Adam Levy,5383 Derrick Turn...,WA,Male,14.054768508881645,Orci Luctus PC,"health, beauty spas",b,4.1,29.102418916409906,11.6173,0.169085794206,0.00110950891487,115.86548880633426,-32.0688145997728,43.0,2509.0,2155.0,2.7,9252.0,18905.0,9660.0,35171.1,32874.3,29.30778218291789,5,4,308,0,3619.1,3842.9
8,2022,3167,2022-08-24,11199348915,5179,433636,830.8376595867408,1e2d8ecf-2586-437...,Adam Long,756 Harris Pine A...,VIC,Undisclosed,13.486187634644764,Orci Luctus PC,"health, beauty spas",b,4.1,57.40148343022908,6.6515,0.120799512286,6.81571981264E-4,145.0942804113976,-37.92438721498632,40.0,2200.0,1915.0,2.7,4905.0,9851.0,4946.0,33877.5,34812.7,12.553937333202164,4,24,236,0,8930.7,8580.0
3,2022,6640,2022-03-31,11199348915,14027,87538,128.8660136536158,bb9ffac1-036d-448...,Adam Malone,47339 Barbara Street,WA,Male,15.160084794148002,Orci Luctus PC,"health, beauty spas",b,4.1,29.00547570250949,20882.7935,11.0582667957,1.89870643878,117.65510568397144,-26.98876645026335,54.0,1333.0,682.0,1.5,147.0,215.0,73.0,32830.9,33559.6,3.1823455149159954,5,31,90,0,367

In [41]:
df.filter(F.col('tags') == 'health, beauty spas')

merchant_abn,tags,take_rate,type,Total Revenue,Total Sales,Merchant Fraud,Consumer Fraud,Median Family Income,avg(seasonal_sales),avg(original_sales),Average Household Size,Unique Customers,Average Expected Revenue,Days with Sales
83935353706,"health, beauty spas",4.22,b,482997.6304497928,2561,42.7456339583728,14.336952436540164,1982.6848887153456,6439.4534947286265,6411.30113237017,2.4640765326044516,2398,3.8924388746637573,594
18146314043,"health, beauty spas",4.41,b,76006.35609335377,273,45.59989528639596,14.404352354243299,1984.3296703296703,6613.262637362638,6634.388278388277,2.4512820512820515,272,5.685108577969417,214
27099208477,"health, beauty spas",6.43,a,286440.28826765023,985,43.180932903261436,14.588572324485916,2036.8,6569.271167512691,6541.734517766499,2.4889340101522843,959,9.164619081252106,479
87788702467,"health, beauty spas",3.58,b,23186.914606357666,107,29.023394512333017,14.252762576817577,1886.0654205607477,6039.1943925233645,5965.742990654205,2.4495327102803737,107,4.714145842101514,101
31471800699,"health, beauty spas",1.57,c,20742.07165888181,170,29.02223359385468,13.756249398810748,1954.629411764706,6740.518823529413,6610.648235294116,2.4323529411764704,168,1.171323724188334,154
76745031694,"health, beauty spas",5.71,a,171647.29652520965,880,49.89273316303949,13.824782753942658,1955.528409090909,6503.806022727272,6487.9922727272715,2.463181818181818,862,4.759010685552576,459
23490868997,"health, beauty spas",1.03,d,284226.5544099753,995,29.028772793075312,13.494019133027008,1923.9125628140703,6439.834974874369,6415.364221105527,2.464321608040201,959,1.8054060712275832,472
17386856164,"health, beauty spas",2.49,c,40726.679679944384,154,29.02227847444757,14.755540698407582,1990.2142857142858,6649.207142857143,6676.8889610389615,2.4974025974025977,152,3.996295461030218,136
41197194464,"health, beauty spas",4.25,b,78402.20872581724,145,42.85663971761808,15.132953302684843,1951.2137931034483,6445.971724137932,6474.5199999999995,2.4213793103448276,144,10.805492770543283,131
74391255526,"health, beauty spas",3.76,b,817001.4595642586,4294,29.02691746601011,14.563048954344076,1989.6399627387052,6534.137121564979,6534.311993479274,2.4542384722869115,3860,4.318127991889822,606
